In [2]:
import sys
sys.path.append("..")
import torch
from common_utils import DATA_HOME
from lstm.sales_data import Sales_Dataset

sd = Sales_Dataset(f"{DATA_HOME}\sales_data")
len(sd)

C:\Users\sxing\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\sxing\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


90936

In [7]:
sd.TR.describe(), sd.TS.describe(), sd.S.describe(), sd.O.describe()

(                 id     store_nbr        family         sales   onpromotion
 count  3.000888e+06  3.000888e+06  3.000888e+06  3.000888e+06  3.000888e+06
 mean   1.500444e+06  2.750000e+01  1.342528e-17  3.579558e-02  4.254511e-02
 std    8.662819e+05  1.558579e+01  1.000000e+00  5.267061e-01  4.635425e-01
 min    0.000000e+00  1.000000e+00 -1.680336e+00 -8.273643e+00 -5.505332e+00
 25%    7.502218e+05  1.400000e+01 -8.401681e-01 -1.177830e-01  0.000000e+00
 50%    1.500444e+06  2.750000e+01  0.000000e+00  0.000000e+00  0.000000e+00
 75%    2.250665e+06  4.100000e+01  8.401681e-01  1.892420e-01  0.000000e+00
 max    3.000887e+06  5.400000e+01  1.680336e+00  8.635093e+00  5.529429e+00,
           store_nbr  transactions
 count  83488.000000  83488.000000
 mean      26.939237     -0.000012
 std       15.608204      0.196840
 min        1.000000     -6.256390
 25%       13.000000     -0.090384
 50%       27.000000     -0.002441
 75%       40.000000      0.096003
 max       54.000000      

In [ ]:
import pandas as pd
OG_TR = pd.read_csv(f"{DATA_HOME}/sales_data/train.csv", index_col=False)
OG_TS = pd.read_csv(f"{DATA_HOME}/sales_data/transactions.csv", index_col=False) 
OG_TR.sort_values(["store_nbr", "family", "date"], inplace=True)
# OG_TR.sales = np.log(OG_TR.sales / OG_TR.sales.shift(1))
OG_TR[200:300], OG_TR[:10], OG_TR[:]